In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

# All requested packages already installed.

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

# All requested packages already installed.

Libraries imported.


### Using Beautiful Soup for Web Scraping

In [9]:
# import libraries
from urllib.request import urlopen as UReq
from bs4 import BeautifulSoup as soup
# specify the url
urlpage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 


# query the website and return the html to the variable 'page'
UClient = UReq(urlpage)

# Opening up the connection and grabbing the page
page_html=UClient.read()


In [10]:
#Load UClient, turn into soup and get the <table>s
#page_html= open('List_of_postal_codes_of_Canada:_M.html').read()

# parse the html using beautiful soup and store in variable 'page_soup'
page_soup = soup(page_html, 'html.parser')

#Scrape out the required table 
tables2=page_soup.find_all('table',class_="wikitable sortable")

"""
# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.findAll('th')
    headings = [th.text for th in ths]
df=pd.DataFrame(columns=headings)
dfa

"""


"\n# Search through the tables for the one with the headings we want.\nfor table in tables:\n    ths = table.findAll('th')\n    headings = [th.text for th in ths]\ndf=pd.DataFrame(columns=headings)\ndfa\n"

In [33]:
#Using prettify to get the html of the required field
#print(page_soup.prettify())

## Exatracting information from web using Html tags

In [12]:
#Exacted headers for the tables
header=[]
for headers in tables2:
    ths=headers.findAll('th')
    for th in ths:
        header.append(th.text)
header

['Postcode', 'Borough', 'Neighbourhood\n']

In [32]:
#Exacting required text fields from the wiki table 
rowValList = []

for i in range(len(page_soup.findAll('td'))):
    rowVal = page_soup.findAll('td')[i].get_text()
    rowValList.append(rowVal)
#rowValList

In [22]:
#separating the text from the above list into seperate lists
postcode = []
for i in range(0, len(rowValList), 3):
    postcode.append(rowValList[i])
    
borough = []
for i in range(1, len(rowValList), 3):
    borough.append(rowValList[i])
    
neigh = []
for i in range(2, len(rowValList), 3):
    neigh.append(rowValList[i])


### Cleaning up the data to match the requirement

In [23]:
#cleaning the data and creating a data frame for each list above
pc_df = pd.DataFrame(postcode)
pc_df.drop(pc_df.tail(12).index,inplace=True)
#print(pc_df)


b_df = pd.DataFrame(borough)
b_df.drop(b_df.tail(11).index,inplace=True)
#b_df

n_df = pd.DataFrame(neigh)#,columns=neigh)
n_df.drop(n_df.tail(11).index,inplace=True)
#n_df

In [24]:
#concatenating all the dataframes column wise into one dataframe called result
final1 = pd.concat([pc_df, b_df,n_df], axis=1, ignore_index=True)


In [25]:
#adding headers to the dataframe
final1.columns=['Postalcode', 'Borough', 'Neighbourhood']


In [26]:
#Striping out unnecesary text from neighbourhood field and the final dataframe is ready to use

final1['Neighbourhood'] = final1['Neighbourhood'].str.rstrip('\n')



In [27]:
#Just duplicating the dataframe before further cleaning so that the original dataframe will not be effected

final2=final1.copy()


### Processing only rows with assigned Borough

In [28]:
# Get names of indexes for which column Neighbourhood has value Not assigned
indexNames = final1[final1['Borough'] == 'Not assigned' ].index
indexNames 
# Delete these row indexes from dataFrame
final1.drop(indexNames , inplace=True)
final1=final1.reset_index(drop=True)


### Grouping the dataset with PostalCode and Borough

In [29]:
#Grouping more than one neighbourhood with same postal codes and Borough
final1 = final1.groupby(['Postalcode','Borough']).agg(lambda x: ", ".join(x.astype(str))).reset_index()


### Assigning the 'Not assigned' values in Neighbourhood with the value in Borough

In [30]:
final1.loc[final1.Neighbourhood == 'Not assigned', 'Neighbourhood'] = final1.Borough


### Shape of final dataset

In [31]:
final1.shape


(103, 3)